# Applicazione dei tagli di Gomory
### Importazione librerie

In [91]:
import numpy as np
import pandas as pd

### Definizione funzioni

- Funzione per restituire il tableau preparato a partire dai parametri

In [92]:
def get_initial_tableau(objective_function,constraints,maximum_values):
    tableau=[]
    objective_function = [-n for n in objective_function]
    tableau.append(objective_function)
    n_constraints = len(constraints)
    n_slack = n_constraints
    index = 0
    for constraint in constraints:
        constraint.insert(0, maximum_values[index])
        for i in range(0,n_slack):
            if i == index:
                constraint.append(1)
            else:
                constraint.append(0)
        index = index + 1
        constraint.append(0)
        tableau.append(constraint)
    for i in range(0,n_slack):
        objective_function.append(0)
    objective_function.insert(0,0)
    objective_function.append(1)
    return tableau

- Funzioni di gestione del pivot. In particolare, tale valore sarà ogni volta individuato tramite:
    - Colonna con il coefficiente della funzione obiettivo minore
    - Riga con il rapporto capacità/valore inferiore (ma comunque maggiore di 0)

In [93]:
def find_min_greater_than_zero(vect):
    min = 0
    for elem in vect:
        if elem > 0 and elem < vect[min]:
            min = vect.index(elem)
    return min
def get_pivot_coordinates(tableau):
    pivot_column = tableau[0].index(min(tableau[0]))
    pivot_column_values=[]
    for row in range(1,len(tableau)):
        if tableau[row][pivot_column] != 0:
            pivot_column_values.append(round(tableau[row][0]/tableau[row][pivot_column],3))
        else:
            pivot_column_values.append(-1)
    pivot_row=find_min_greater_than_zero(pivot_column_values)+1
    return pivot_row, pivot_column

- Funzione iterativa, che restituisce il tableau successivo a partire da uno in ingresso.
    - Per ogni riga diversa da quella del pivot, somma ad essa la riga del pivot moltiplicata per un coefficiente tale da azzerare il valore sulla stessa colonna del pivot
- La seconda funzione controlla se il tableau è quello finale: è tale se tutti i coefficienti della funzione obiettivo sono positivi

In [94]:
def nextStep(tableau):
    pivot_row,pivot_column = get_pivot_coordinates(tableau)
    for row in range(0,len(tableau)):
        if row != pivot_row:
            multiplier = tableau[row][pivot_column]/tableau[pivot_row][pivot_column]
            for index in range(0,len(tableau[0])):
                tableau[row][index] = round(tableau[row][index] - multiplier*tableau[pivot_row][index],3)
    return tableau

def check_continue(tableau):
    CONTINUE = False
    for elem in tableau[0]:
        if elem < 0:
            CONTINUE = True
    return CONTINUE

- Funzione che calcola le soluzioni ed il valore ottimo a partire dal tableau finale. Per calcolare i valori della soluzione bisogna prendere in considerazione tutte le colonne (non di slack), con un solo valore diverso da 0, dividendo tale valore per il primo termine a sinistra della stessa riga.

In [95]:
def get_solutions(tableau, items):
    cols = []
    solutions=[]
    for r in range(0,items+1):
        cols.append([row[r] for row in tableau])
    #Arrivato qui ho l'insieme delle colonne. Se in qualcuna c'è un solo elemento diverso da 0, la prendo
    for col in cols:
        k = 0
        sol=0
        for i in col:
            if i != 0:
                k = k+1
                row = col.index(i)
        if k == 1:
             #Sto osservando una colonna soluzione
             sol = tableau[row][0]/tableau[row][cols.index(col)]
        solutions.append([sol,row])
    opt = tableau[0][0]
    return solutions[1:],opt

- Funzione che controlla che le soluzioni trovate non siano sovrapposte sugli stessi vincoli. Qualora lo fossero, vengono divise

In [96]:
def check_multiple_sols(sols,optimum,objective_function):
    #Vediamo cosa fare quando ci sono più soluzioni ottime
    equalities=[]
    solutions =[]
    for s in range(0,len(sols)-1):
        for s2 in range(s+1,len(sols)):
            if sols[s][1] == sols[s2][1] and objective_function[s]*sols[s][0] == objective_function[s2]*sols[s2][0] and sols[s][0]!=0:
                print("Multiple solutions found.")
                equalities.append([s,s2])
    if equalities:
        sol1=[]
        sol2=[]
        for e in equalities:
            for s in sols:
                if sols.index(s) == e[1]:
                    sol1.append(0)
                else:
                    sol1.append(sols[sols.index(s)][0])
            for s in sols:
                if sols.index(s) == e[0]:
                    sol2.append(0)
                else:
                    sol2.append(sols[sols.index(s)][0])
            solutions.append(sol1)
            solutions.append(sol2)
    return solutions

### Definizione dei parametri del problema

In [97]:
objective_function = [3,2,1,4]
weights = [3,2,1,6,]
volumes = [3,5,1,6]
max_weight = 7
max_volume = 7
items = len(objective_function)

of = objective_function.copy()
w = weights.copy()
v = volumes.copy()

TABLEAU = get_initial_tableau(of,[w,v],[max_weight,max_volume])
print(pd.DataFrame(TABLEAU))

   0  1  2  3  4  5  6  7
0  0 -3 -2 -1 -4  0  0  1
1  7  3  2  1  6  1  0  0
2  7  3  5  1  6  0  1  0


### Risoluzione del tableau

In [98]:
#Fino a quando non avremo la riga 0 non avrà tutti valori positivi, si itera la generazione dei tableau.
print("INITIAL TABLEAU:\n")
print(pd.DataFrame(TABLEAU))
while check_continue(TABLEAU):
    print("\n----RESOLUTION STEP----\n")
    pivot_row,pivot_column = get_pivot_coordinates(TABLEAU)
    print("The pivot is in position[",pivot_row,",",pivot_column,"]")
    print("Tableau after the step execution:\n")
    nextStep(TABLEAU)
    print(pd.DataFrame(TABLEAU))

INITIAL TABLEAU:

   0  1  2  3  4  5  6  7
0  0 -3 -2 -1 -4  0  0  1
1  7  3  2  1  6  1  0  0
2  7  3  5  1  6  0  1  0

----RESOLUTION STEP----

The pivot is in position[ 1 , 4 ]
Tableau after the step execution:

       0    1      2      3    4      5    6    7
0  4.667 -1.0 -0.667 -0.333  0.0  0.667  0.0  1.0
1  7.000  3.0  2.000  1.000  6.0  1.000  0.0  0.0
2  0.000  0.0  3.000  0.000  0.0 -1.000  1.0  0.0

----RESOLUTION STEP----

The pivot is in position[ 1 , 1 ]
Tableau after the step execution:

     0    1    2    3    4    5    6    7
0  7.0  0.0 -0.0  0.0  2.0  1.0  0.0  1.0
1  7.0  3.0  2.0  1.0  6.0  1.0  0.0  0.0
2  0.0  0.0  3.0  0.0  0.0 -1.0  1.0  0.0


### Stampa dei risultati

In [99]:
solution, optimum = get_solutions(TABLEAU,items)
#Il solutions vector rappresenta l'unione anche di soluzioni sovrapposte, la struttura è [valore soluzione,vincolo relativo]
multiple_solutions=check_multiple_sols(solution,optimum,objective_function)
if multiple_solutions: 
    print("There are equivalent solutions.")
    print(multiple_solutions)
else:
    print("Solutions vector: ",solution)
print("\nThe optimum value is ",optimum)

Multiple solutions found.
There are equivalent solutions.
[[2.3333333333333335, 0, 0, 0], [0, 0, 7.0, 0]]

The optimum value is  7.0
